In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.master("local[1]") \
                    .appName('test') \
                    .getOrCreate()

23/08/17 06:52:42 WARN Utils: Your hostname, codespaces-d00206 resolves to a loopback address: 127.0.0.1; using 172.16.5.4 instead (on interface eth0)
23/08/17 06:52:42 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/08/17 06:52:44 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# Datasets: Type-Safe Structured APIs
The first API we’ll describe is a type-safe version of Spark’s structured API called Datasets, for writing statically typed code in Java and Scala. The Dataset API is not available in Python and R, because those languages are dynamically typed.

# Structured Streaming

Let’s first analyze the data as a static dataset and create a DataFrame to do so.
We’ll also create a schema from this static dataset

In [4]:
staticDataFrame = spark.read.format('csv')\
    .option('header', 'true')\
    .option('inferSchema', 'true')\
    .load('../data/retail-data/by-day/*.csv')

In [5]:
staticDataFrame.createOrReplaceTempView('retail_data')
staticSchema = staticDataFrame.schema

The window function will include all data from each day in the aggregation. It’s simply a window over the time–series column in our data. This is a helpful tool for manipulating date and timestamps because we can specify our requirements in a more human form (via intervals), and Spark will group all of them together for us:

In [7]:
from pyspark.sql.functions import window, column, desc, col
staticDataFrame\
    .selectExpr(
        'CustomerId',
        '(UnitPrice * Quantity) as total_cost',
        'InvoiceDate')\
    .groupBy(
        col('CustomerId'), window(col('InvoiceDate'), '1 day'))\
    .sum('total_cost')\
    .show(5)

+----------+--------------------+-----------------+
|CustomerId|              window|  sum(total_cost)|
+----------+--------------------+-----------------+
|   16057.0|{2011-12-05 00:00...|            -37.6|
|   14126.0|{2011-11-29 00:00...|643.6300000000001|
|   13500.0|{2011-11-16 00:00...|497.9700000000001|
|   17160.0|{2011-11-08 00:00...|516.8499999999999|
|   15608.0|{2011-11-11 00:00...|            122.4|
+----------+--------------------+-----------------+
only showing top 5 rows



Because you’re likely running this in local mode, it’s a good practice to set the number of shuffle partitions to something that’s going to be a better fit for local mode. This configuration specifies the number of partitions that should be created after a shuffle. By default, the value is 200, but because there aren’t many executors on this machine, it’s worth reducing this to 5.

In [8]:
spark.conf.set('spark.sql.shuffle.partitions', '5')

Now that we’ve seen how that works, let’s take a look at the streaming code! You’ll notice that very little actually changes about the code. The biggest change is that we used readStream instead of read, additionally you’ll notice the maxFilesPerTrigger option, which simply specifies the number of files we should read in at once. This is to make our demonstration more “streaming,” and in a production scenario this would probably be omitted.

In [9]:
streamingDataFrame = spark.readStream\
    .schema(staticSchema)\
    .option('maxFilesPerTrigger', 1)\
    .format('csv')\
    .option('header', 'true')\
    .load('../data/retail-data/by-day/*.csv')

In [10]:
streamingDataFrame.isStreaming

True

Let’s set up the same business logic as the previous DataFrame manipulation. We’ll perform a
summation in the process:

In [12]:
purchaseByCustomerPerHour = streamingDataFrame\
    .selectExpr(
        'CustomerId',
        '(UnitPrice * Quantity) as total_cost',
        'InvoiceDate')\
    .groupBy(
        col('CustomerId'), window(col('InvoiceDate'), '1 day'))\
    .sum('total_cost')

This is still a lazy operation, so we will need to call a streaming action to start the execution of
this data flow.
Streaming actions are a bit different from our conventional static action because we’re going to be populating data somewhere instead of just calling something like count (which doesn’t make any sense on a stream anyways). The action we will use will output to an in-memory table that we will update after each trigger. In this case, each trigger is based on an individual file (the read option that we set). Spark will mutate the data in the in-memory table such that we will always have the highest value as specified in our previous aggregation:

In [13]:
purchaseByCustomerPerHour.writeStream\
    .format('memory')\
    .queryName('customer_purchases')\
    .outputMode('complete')\
    .start()

23/08/17 07:57:40 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-946e8a07-7575-4b53-9ecc-bd2d255665e6. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
23/08/17 07:57:40 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


23/08/17 07:57:44 WARN FileStreamSource: Listed 305 file(s) in 3560 ms          


When we start the stream, we can run queries against it to debug what our result will look like if
we were to write this out to a production sink:

In [14]:
spark.sql('''
SELECT *
FROM customer_purchases
ORDER BY `sum(total_cost)` DESC
''')\
.show(5)

+----------+--------------------+------------------+
|CustomerId|              window|   sum(total_cost)|
+----------+--------------------+------------------+
|      null|{2011-03-29 00:00...| 33521.39999999998|
|      null|{2010-12-21 00:00...|31347.479999999938|
|   18102.0|{2010-12-07 00:00...|          25920.37|
|      null|{2010-12-10 00:00...|25399.560000000012|
|      null|{2010-12-17 00:00...|25371.769999999768|
+----------+--------------------+------------------+
only showing top 5 rows



You’ll notice that the composition of our table changes as we read in more data! With each file, the results might or might not be changing based on the data. Naturally, because we’re grouping customers, we hope to see an increase in the top customer purchase amounts over time (and do for a period of time!).